In [1]:
import pandas as pd
import numpy as np


In [103]:
filename = "./data/text_classify_1.csv"

In [104]:
df = pd.read_csv(filename)

In [105]:
df.head()

,target,context
0,买房,房子还在吗
1,买房,什么时候可以看房？
2,买房,您好，请问这套房子还在吗？
3,买房,你好，我想在浦东新场附近买个商住两用的房子
4,买房,我是想全款买套房，560左右，想找个不满二税费高的房子，这样我买就很划算


In [106]:
print("在 target 列中总共有 %d 个空值." % df['target'].isnull().sum())
print("在 context 列中总共有 %d 个空值." % df['context'].isnull().sum())
df[df.isnull().values==True]
df = df[pd.notnull(df['context'])]

在 cat 列中总共有 0 个空值.
在 review 列中总共有 0 个空值.


In [107]:
print("在 target 列中总共有 %d 个空值." % df['target'].isnull().sum())
print("在 context 列中总共有 %d 个空值." % df['context'].isnull().sum())

在 cat 列中总共有 0 个空值.
在 review 列中总共有 0 个空值.


In [108]:
df['target_id'] = df['target'].factorize()[0]
cat_id_df = df[['target', 'target_id']].drop_duplicates().sort_values('target_id').reset_index(drop=True)
cat_to_id = dict(cat_id_df.values)
id_to_cat = dict(cat_id_df[['target_id', 'target']].values)
df.sample(5)

,target,context,target_id
43,买房,我要买楼,0
738,租赁,计划租一个大约6350块钱左右地,1
860,咨询,2019年满五年的房子过户要交多少税？,2
287,买房,计划买个预估900萬元RMB毛坯房,0
223,买房,买个大约450萬元左右毛坯房产,0


In [109]:
cat_id_df

,target,target_id
0,买房,0
1,租赁,1
2,咨询,2


In [110]:

import jieba
import re
jieba.load_userdict('./data/dict.txt')
#定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
 
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  
 
#加载停用词
stopwords = stopwordslist("./data/stop_words.txt")

In [111]:
#删除除字母,数字，汉字以外的所有符号
df['clean_content'] = df['context'].apply(remove_punctuation)
df.sample(10)

,target,context,target_id,clean_content
888,咨询,5年以下的二套房转卖卖房交多少税？,2,5年以下的二套房转卖卖房交多少税
139,买房,我想看总价1000萬元上下得新房,0,我想看总价1000萬元上下得新房
271,买房,我想mai大约W以里精装的新房,0,我想mai大约W以里精装的新房
987,咨询,采用委托收款方式缴存的单位，能否一次托收多个月份的住房公积金？,2,采用委托收款方式缴存的单位能否一次托收多个月份的住房公积金
1133,咨询,如何登录房改售房款和房改房维修资金网上业务系统？,2,如何登录房改售房款和房改房维修资金网上业务系统
227,买房,我想要买价值250精装二手房,0,我想要买价值250精装二手房
711,租赁,租房预估4500块钱上下次卧屋子,1,租房预估4500块钱上下次卧屋子
644,租赁,我想月底再租,1,我想月底再租
609,租赁,长租的那种,1,长租的那种
574,租赁,想找一找租房63507000以下上下精装得房子,1,想找一找租房63507000以下上下精装得房子


In [112]:
#分词，并过滤停用词
df['cut_context'] = df['clean_content'].apply(lambda x: " ".join([w for w in list(jieba.cut(x)) if w not in stopwords]))
df.tail()

,target,context,target_id,clean_content,cut_context
1182,咨询,你的城市房租收入比是多少？,2,你的城市房租收入比是多少,城市 房租 收入 比是
1183,咨询,北京老人能投靠子女落户吗？怎么办理？,2,北京老人能投靠子女落户吗怎么办理,北京 老人 投靠 子女 落户 办理
1184,咨询,北京能办理夫妻投靠落户吗？怎么办理？,2,北京能办理夫妻投靠落户吗怎么办理,北京 办理 夫妻 投靠 落户 办理
1185,咨询,2017年环京限购政策一览,2,2017年环京限购政策一览,2017 年 环京 限购 政策 一览
1186,咨询,买商铺需要什么资格，要注意什么？,2,买商铺需要什么资格要注意什么,买 商铺 资格


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(norm='l2', ngram_range=(1, 3))
features = tfidf.fit_transform(df.cut_context)
labels = df.target_id
print(features.shape)
# print('-----------------------------')
# print(features)

NameError: name 'df' is not defined

In [114]:
cat_id_df

,target,target_id
0,买房,0
1,租赁,1
2,咨询,2


In [115]:
from sklearn.feature_selection import chi2
import numpy as np
 
N = 50
for target, target_id in sorted(cat_to_id.items()):
    features_chi2 = chi2(features, labels == target_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    tregrams = [v for v in feature_names if len(v.split(' ')) == 3]
    print("# '{}':".format(target))
    
    print("  . Most correlated unigrams:\n       . {}".format('\n        '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n       . {}".format('\n        '.join(bigrams[-N:])))
    print("  . Most correlated trerams:\n       . {}".format('\n        '.join(tregrams[-N:])))

# '买房':
  . Most correlated unigrams:
       . 400wrmb
        800
        想要
        学区
        750w
        zu
        块钱
        zufang
        635
        900w
        提取
        750
        一点
        租个
        请问
        150
        350
        办理
        250
        950
        精装
        100
        900
        整租
        钱能
        550
        700
        简装
        500
        450
        万左右
        买房子
        300
        850
        200
        总价
        毛坯
        租房
        我要
        住房
        公积金
        650
        以里
        买个
        mai
        maifang
        万元
        萬元
        rmb
        人民币
  . Most correlated bigrams:
       . 900 萬元
        900 万元
        精装 二手房
        想要 maifang
        买个 价钱
        想要 mai
        公积金 住房贷款
        人民币 新房
        买个 估计
        萬元 毛坯
        200 万元
        mai 估计
        maifang 价值
        计划 maifang
        maifang 总价
        maifang 价钱
        万元 毛坯
        万元 精装
        总价 500
        maifang 估计
        计划 买个
     

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df['cut_context'], df['target_id'],test_size=0.1,random_state = 0)
#count_vect = CountVectorizer()
#X_train_counts = count_vect.fit_transform(X_train)
 
#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
vec = TfidfVectorizer(norm='l2', ngram_range=(1, 3))
X_train_tf= vec.fit_transform(X_train)
nb_clf = MultinomialNB(alpha=0.1,fit_prior=True,class_prior=None)
nb_clf.fit(X_train_tf, y_train)
nb_y_pred = nb_clf.predict(vec.transform(X_test))

In [117]:
def nb_myPredict(sec):
    format_sec=" ".join([w for w in list(jieba.cut(remove_punctuation(sec))) if w not in stopwords])
    pred_cat_id=nb_clf.predict(vec.transform([format_sec]))
    print(id_to_cat[pred_cat_id[0]])

In [120]:
nb_myPredict("租")

买房


In [121]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
print('accuracy %s' % accuracy_score(nb_y_pred, y_test))
print(classification_report(y_test, nb_y_pred,target_names=cat_id_df['target'].values))

accuracy 0.9243697478991597
              precision    recall  f1-score   support

          买房       0.82      0.94      0.88        34
          租赁       1.00      0.85      0.92        46
          咨询       0.95      1.00      0.97        39

    accuracy                           0.92       119
   macro avg       0.92      0.93      0.92       119
weighted avg       0.93      0.92      0.92       119



In [122]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC


# 逻辑回归

In [123]:
lr_clf = LogisticRegression(C=1.0, 
                            class_weight='balanced', 
                            dual=False, 
                            fit_intercept=False,
                            max_iter=10000,
                            multi_class='ovr', 
                            n_jobs=-1, 
                            penalty='l2', 
                            solver='lbfgs',
                            random_state=0, 
                            tol=0.0001)

In [124]:
lr_clf.fit(X_train_tf, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=False, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='ovr', n_jobs=-1, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [125]:
lr_y_pred = lr_clf.predict(vec.transform(X_test))

In [126]:
print('accuracy %s' % accuracy_score(lr_y_pred, y_test))
print(classification_report(y_test, lr_y_pred,target_names=cat_id_df['target'].values))

accuracy 0.907563025210084
              precision    recall  f1-score   support

          买房       0.78      0.94      0.85        34
          租赁       0.97      0.83      0.89        46
          咨询       0.97      0.97      0.97        39

    accuracy                           0.91       119
   macro avg       0.91      0.91      0.91       119
weighted avg       0.92      0.91      0.91       119



In [127]:
def lr_myPredict(sec):
    format_sec=" ".join([w for w in list(jieba.cut(remove_punctuation(sec))) if w not in stopwords])
    pred_cat_id=lr_clf.predict(vec.transform([format_sec]))
    print(id_to_cat[pred_cat_id[0]])

In [128]:
lr_myPredict('租')

买房


# 支持向量机

In [129]:
sv_clf = LinearSVC(C=0.1,                                     
                   class_weight=None,       
                   fit_intercept=False,       
                   loss='squared_hinge',    
                   max_iter=10000,
                   multi_class='ovr',       
                   penalty='l2',        
                   random_state=2019, 
                   tol=0.0001)

In [130]:
sv_clf.fit(X_train_tf, y_train)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=False,
          intercept_scaling=1, loss='squared_hinge', max_iter=10000,
          multi_class='ovr', penalty='l2', random_state=2019, tol=0.0001,
          verbose=0)

In [131]:
sv_y_pred = sv_clf.predict(vec.transform(X_test))

In [132]:
print('accuracy %s' % accuracy_score(sv_y_pred, y_test))
print(classification_report(y_test, sv_y_pred,target_names=cat_id_df['target'].values))

accuracy 0.907563025210084
              precision    recall  f1-score   support

          买房       0.78      0.94      0.85        34
          租赁       0.97      0.83      0.89        46
          咨询       0.97      0.97      0.97        39

    accuracy                           0.91       119
   macro avg       0.91      0.91      0.91       119
weighted avg       0.92      0.91      0.91       119



In [133]:
def sv_myPredict(sec):
    format_sec=" ".join([w for w in list(jieba.cut(remove_punctuation(sec))) if w not in stopwords])
    pred_cat_id=sv_clf.predict(vec.transform([format_sec]))
    print(id_to_cat[pred_cat_id[0]])

In [134]:
sv_myPredict("租")

买房


# xgboost

In [89]:
xgb_clf = XGBClassifier(learning_rate=0.01,        #学习率
                        n_estimators=1000,          # 树的个数--1000棵树建立xgboost
                        max_depth=6,             # 树的深度
                        min_child_weight=1,        # 叶子节点最小权重
                        gamma=0.,                  # 惩罚项中叶子结点个数前的参数
                        subsample=0.8,             # 随机选择80%样本建立决策树
                        colsample_btree=0.8,       # 随机选择80%特征建立决策树
                        objective='multi:softmax', # 指定损失函数
                        colsample_bynode=1,        #每个节点（拆分）的列的子样本比率
                        colsample_bylevel=1,       #构建每一层时，列采样率 
                        nthread=20,                #并行线程数
                        random_state=2019)

In [90]:
xgb_clf.fit(X_train_tf,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_btree=0.8,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0.0, learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=20, objective='multi:softprob', random_state=2019,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [91]:
xgb_y_pred = xgb_clf.predict(vec.transform(X_test))

In [92]:
print('accuracy %s' % accuracy_score(xgb_y_pred, y_test))
print(classification_report(y_test, xgb_y_pred,target_names=cat_id_df['target'].values))

accuracy 0.7563025210084033
              precision    recall  f1-score   support

          买房       0.78      0.74      0.76        34
          租赁       0.91      0.63      0.74        46
          咨询       0.65      0.92      0.77        39

    accuracy                           0.76       119
   macro avg       0.78      0.76      0.76       119
weighted avg       0.79      0.76      0.75       119



In [93]:
def xgb_myPredict(sec):
    format_sec=" ".join([w for w in list(jieba.cut(remove_punctuation(sec))) if w not in stopwords])
    pred_cat_id=xgb_clf.predict(vec.transform([format_sec]))
    print(id_to_cat[pred_cat_id[0]])

In [95]:
xgb_myPredict("租")

咨询
